#######################################################################################################

Executes daly to update all the stock signals 
Current signals -
1. EMA Cross indicator
2. Turn Around Stocks

#######################################################################################################


In [2]:
import os, sys

# Get repo root (works both locally & on GitHub Actions)
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
src_path = os.path.join(repo_root, "src")

if src_path not in sys.path:
    sys.path.append(src_path)

print("✅ Added to path:", src_path)
print("Current working directory:", os.getcwd())

# Now your imports will work
import get_stocks_data as gs

# from src import __version__
# print(__version__)

✅ Added to path: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\src
Current working directory: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\notebooks


In [3]:
import numpy as np 
import pandas as pd 
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',120)
import datetime as dt
import datetime
import time 
import ta
import sqlite3

In [4]:
analysis_period = 30 # past 30 days for Analysis
raw_data_needed = analysis_period + 1 * 365  # +1 year of buffer for calculating indicators
output_period = 30 # restrict output file for EMA-signals in 30 days

In [5]:
# Determine repository root
# GitHub sets GITHUB_WORKSPACE; locally, we use current working directory
repo_root = os.environ.get("GITHUB_WORKSPACE", os.getcwd())

# if repo_root == os.getcwd(): #then go one directory above to reach repo root
#     repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

if "d:\\Work" in repo_root or "D:\\Work" in repo_root:
    repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

print("Repository root determined as:", repo_root)

# Construct full path to the database
db_path = os.path.join(repo_root, "data", "stocks.db")

# Optional: check if it exists
if not os.path.exists(db_path):
    print(f"ERROR: Database not found at {db_path}")
    sys.exit(1)  # stop execution if DB missing

print("Using database at:", db_path)

# Now you can safely call your function
# Example:
# stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')


Repository root determined as: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals
Using database at: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\data\stocks.db


In [6]:

stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')
stocklist.head()

tickers = list(stocklist.ticker.unique())
print(len(tickers))
# tickers = ["ESSARSHPNG.NS","LATENTVIEW.NS","BHEL.NS"]

stock_df = gs.fetch_stock_data(tickers, raw_data_needed)

stock_df.head()

2025-11-09 23:49:41,078 - INFO - 📈 Fetching data for 2178 tickers...
2025-11-09 23:49:41,079 - INFO - Fetching data from 2024-10-10 to 2025-11-09


d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\data\stocks.db
2178


[*********************100%***********************]  2178 of 2178 completed
2025-11-09 23:52:48,265 - ERROR - 
1 Failed download:
2025-11-09 23:52:48,269 - ERROR - ['NDTV-RE.NS']: YFInvalidPeriodError("NDTV-RE.NS: Period 'max' is invalid, must be of the format 1d, 5d, etc.")
2025-11-09 23:52:53,584 - INFO - ✅ Download completed in 192.51 seconds
2025-11-09 23:53:00,514 - INFO - 📊 Final dataset: 559,933 rows from 2024-10-10 to 2025-11-07


Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,20ema,50ema,100ema,200ema,30dv,7dv,Symbol
0,2024-10-10,20MICRONS.NS,NaN,267.840363,278.044279,264.256572,277.347445,38454.0,267.840363,267.840363,267.840363,267.840363,NaN,NaN,20MICRONS
1,2024-10-11,20MICRONS.NS,NaN,263.808594,273.912955,259.876340,267.840373,37552.0,267.456385,267.682254,267.760526,267.800245,NaN,NaN,20MICRONS
2,2024-10-14,20MICRONS.NS,NaN,265.301849,273.763634,260.573205,273.763634,66008.0,267.251191,267.588905,267.711839,267.775386,NaN,NaN,20MICRONS
3,2024-10-15,20MICRONS.NS,NaN,263.609467,269.482965,258.831059,266.197783,54722.0,266.904360,267.432848,267.630604,267.733934,NaN,NaN,20MICRONS
4,2024-10-16,20MICRONS.NS,NaN,276.750122,276.750122,276.750122,276.750122,24756.0,267.842052,267.798232,267.811188,267.823647,NaN,NaN,20MICRONS


##### Call Accumulation signal

In [6]:
import importlib
import accumulation_signal as acs
importlib.reload(acs)

# Detect accumulation
accumulation_df = acs.detect_gradual_accumulation(stock_df)

# Filter the ones meeting entry criteria
accum_df = acs.filter_accumulation(accumulation_df)


status  = acs.save_accumulation_to_db(accum_df, db_path)
if status:
    print("✅ Database updated successfully")
else:
    print("⚠️ Database Update failed")
# View
accum_df.head()

✅ Database updated successfully


,Symbol,Date,AD_Slope,Above_20EMA_%,Avg_Volume_Spike,RSI,Accumulating,20ema,50ema,200ema,Close,7dv,30dv
0,HUHTAMAKI,2025-10-24,257479.0278,85.0,1.30,70.94,True,247.637933,236.510412,238.301213,265.140015,3.329077e+06,9.714275e+05
1,CENTRALBK,2025-10-24,140672.7339,80.0,1.22,58.09,True,37.930796,37.399357,40.914078,39.070000,1.296222e+07,7.975041e+06
2,COASTCORP,2025-10-24,53964.9821,80.0,1.59,75.57,True,37.591339,36.280382,39.009432,42.840000,9.384134e+05,3.433010e+05
3,DIAMONDYD,2025-10-24,25613.4288,80.0,0.81,58.28,True,1031.158168,1017.889998,1042.609551,1037.500000,1.380521e+05,9.066520e+04


#### EMA Signal indicator 


In [18]:
import importlib
import ema_signal as emasignal
importlib.reload(emasignal)

# Detect Golden/Death cross with volume confirmation

long_stocks, short_stocks, ema_df = emasignal.generate_ema_signals(
    stock_df,
    analysis_period=15,
    vol_thresh=1.5,
    output_period=30
)
long_stocks.head()



status  = emasignal.save_EMA_signal_to_db(long_stocks, db_path)
if status:
    print("✅ Database updated successfully")
else:
    print("⚠️ Database Update failed")
# View
ema_df.head()

178 tickers with Close/200ema crossover between 2025-10-26–2025-11-10
38 tickers with 50ema/200ema crossover between 2025-10-26–2025-11-10
✅ Metadata fetched for 205 total stocks
✅ Final signals: 138 long | 82 short
✅ Saved 138 new EMA signal records to SIGNAL_EMA_CROSS
📦 Table now has 3609 total records.
✅ Database updated successfully


,Symbol,signal_1,Date_1,Price_1,7dratio_1,30dratio_1,Vol1,Mom1,signal_2,Date_2,Price_2,7dratio_2,30dratio_2,Vol2,Mom2,currentPrice,Capitalization,MCap,upFrom52wlow,PE,PB,PBXPE,debtToEquity,PS,PricetoCash,EPS,CashperShare,update date,beta,AD,AD_50
0,3MINDIA,Price_gt_200ema,2025-11-03,30700.0,4.29,1.55,4.29,-2.0,Golden cross,2025-11-04,35885.0,5.91,3.38,5.91,29.0,35440.0,Large Cap,39923.0,0.86,72.0,25.0,1789.0,0.06,8.40,64.448785,491.0,550.0,2025-11-10,-0.05,0.69,0.98
1,A2ZINFRA,Price_gt_200ema,2025-11-07,18.0,2.38,1.35,2.38,-1.0,,,NaN,NaN,NaN,NaN,NaN,18.0,Nano Cap,322.0,0.41,33.0,7.0,235.0,3.45,0.94,66.545455,1.0,0.0,2025-11-10,0.17,NaN,NaN
2,AARVEEDEN,Price_gt_200ema,2025-11-06,143.0,0.79,2.14,2.14,-24.0,,,NaN,NaN,NaN,NaN,NaN,151.0,Nano Cap,390.0,0.71,19.0,10.0,184.0,2.64,13.75,26.197232,8.0,6.0,2025-11-10,0.34,NaN,NaN
3,ABBOTINDIA,,,NaN,NaN,NaN,NaN,NaN,Death cross,2025-10-31,28995.0,1.95,0.86,1.95,-33.0,28790.0,Large Cap,61641.0,0.30,42.0,15.0,644.0,0.05,9.16,47.824238,682.0,602.0,2025-11-10,0.00,-0.77,-0.51
4,ADANIENT,Price_gt_200ema,2025-10-29,2537.0,2.22,0.47,2.22,20.0,,,NaN,NaN,NaN,NaN,NaN,2375.0,Large Cap,287057.0,0.38,36.0,5.0,189.0,1.78,3.09,22.711884,67.0,105.0,2025-11-10,0.55,NaN,NaN


##### Update the Stock returns table

In [19]:
# Update the signal returns 
# import accumulation_signal as acs

import importlib
import signal_returns as sigret
# importlib.reload(sigret)

# from signal_returns import generate_signal_returns

status, final_returns = sigret.generate_signal_returns(db_path, stock_df)

if status:
    print("✅ Signal returns generation successful!")
    display(final_returns.head())
else:
    print("❌ Signal returns generation failed.")


✅ Total signals loaded: 1906
✅ Total signals after merging: 281213
📦 Existing SIGNAL_RETURNS records: 1913
✅ New unique records found: 260
💾 SIGNAL_RETURNS table updated successfully.
🔒 Database connection closed.
✅ Signal returns generation successful!


,Symbol,Signal_Type,Signal_date,Signal_Price,Signal_Rank,ret_1w,ret_1w_price,ret_1w_date,ret_2w,ret_2w_price,ret_2w_date,ret_1m,ret_1m_price,ret_1m_date,ret_3m,ret_3m_price,ret_3m_date,ret_6m,ret_6m_price,ret_6m_date,ret_1y,ret_1y_price,ret_1y_date,ret_sinceSignal,current_price,current_date,ret_1w_perc,ret_2w_perc,ret_1m_perc,ret_3m_perc,ret_6m_perc,ret_1y_perc,ret_sinceSignal_perc
1,20MICRONS,Golden cross,2025-07-16,259.0,2,6.86,276.779999,2025-07-23,-5.46,244.850006,2025-07-30,-12.20,227.399994,2025-08-18,-17.18,214.500000,2025-10-14,NaN,NaN,NaT,NaN,NaN,NaT,-17.68,213.210007,2025-11-07,6.86,-5.46,-12.2,-17.18,<NA>,<NA>,-17.68
7,3MINDIA,Golden cross,2025-11-04,35885.0,1,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,-0.85,35580.000000,2025-11-07,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-0.85
11,AARTIDRUGS,Golden cross,2025-07-02,484.0,3,-1.37,477.350006,2025-07-09,7.49,520.250000,2025-07-16,1.13,489.450012,2025-08-01,0.52,486.500000,2025-09-30,NaN,NaN,NaT,NaN,NaN,NaT,1.54,491.450012,2025-11-07,-1.37,7.49,1.13,0.52,<NA>,<NA>,1.54
37,AHLUCONT,Golden cross,2025-07-25,1011.0,6,0.49,1015.982849,2025-08-01,-5.21,958.317932,2025-08-08,-5.21,958.317932,2025-08-25,-9.17,918.250000,2025-10-23,NaN,NaN,NaT,NaN,NaN,NaT,-12.01,889.549988,2025-11-07,0.49,-5.21,-5.21,-9.17,<NA>,<NA>,-12.01
42,AJMERA,Golden cross,2025-06-11,955.0,13,1.46,968.977173,2025-06-18,-2.92,927.137634,2025-06-25,-5.90,898.680725,2025-07-11,5.36,1006.150024,2025-09-09,NaN,NaN,NaT,NaN,NaN,NaT,7.01,1021.900024,2025-11-07,1.46,-2.92,-5.9,5.36,<NA>,<NA>,7.01
